# Observing Anomalies

First of all, you need to make sure that our datastore service is up and running.

In [1]:
init()

22:37:30 Antidote session created. Connected to Antidote node 1.

In this section, we will see how eventually consistent data stores behave when updates are done on disconnected replicas.
 We will demonstrate this by using two replicated DC nodes: replica 1 and replica 2. By default, this notebook connects to replica 1.

<img src="images/Antidote-Notebook-diagram.png">

We first create a Kanban board on replica 1.

In [4]:
onReplica(1)
b  = createboard("b")
c1 = addcolumn(b,"todo")
c2 = addcolumn(b,"done")
t1 = createtask(c1,"Slides")
read(b)read(b)

In [4]:
createboard()

The two replicas are currently connected. Any updates to either one of them will be replicated to the other one. Now, we switch to replica 2 to see if the board is there available.

In [4]:
onReplica(2)

22:39:57 Antidote 2 session created.  Connected to Antidote 2.

In [4]:
read(b)

No such property: b for class: Script9

## Classical Conflict-Resolution

## Inconsistency resolution

### Last-writer-wins

The Last-writer-wins register uses this mechanism. If there are multiple updates while the replicas are disconnected, the final value of the object will be the value updated at the last timestamp.

To simulate this situation, we first create a Last-Writer-Wins register (LWW) key while the replicas are connected.

In [3]:
connect()
lww_key = getLWWRegisterKey("lww1")
applyUpdate(assignLWWRegister(lww_key, "A"))
read(lww_key)

A

Next, we disconnect the nodes.

In [4]:
disconnect()

16:16:50 Disconnecting Antidote nodes.

In [4]:
onReplica(1)
applyUpdate(assignLWWRegister(lww_key, "B"))
read(lww_key)

java.net.SocketException: Broken pipe (Write failed)

In [5]:
onReplica(2)
applyUpdate(assignLWWRegister(lww_key, "C"))
read(lww_key)

C

While `Antidote1` and `Antidote2` are disconnected, key `lww1` was updated differently. This causes an inconsistency to key `lww1`.

In [6]:
connect()

16:17:15 Connecting Antidote nodes.

After re-connecting, Antidote resolves the inconsistencies applying the LWW scheme. The expected value stored in key `lww1` is `"C"` since it is the last update performed on key `lww1`.

In [6]:
onReplica(1)
read(lww_key)

java.net.SocketException: Broken pipe (Write failed)

### Multi-value

If there are multiple updates while Antidote nodes are disconnected, all of the last updates on different Antidote nodes will be included the final values. The multi-value uses this mechanism. The final list is ordered in alphabetical order. In its initial state, reading the register returns the empty list.

In [ ]:
connect()
mv_key = getMVRegisterKey("mv1")
applyUpdate(assignMVRegister(mv_key, "A"))
read(mv_key)

In [ ]:
disconnect()

In [ ]:
onReplica(1)
applyUpdate(assignMVRegister(mv_key, "B"))

In [ ]:
onReplica(2)
applyUpdate(assignMVRegister(mv_key, "C"))
applyUpdate(assignMVRegister(mv_key, "D"))

In [ ]:
connect()

The expected values stored on key mv1 is `["B","D"]` since both were the last update on different Antidote nodes during disconnection.

In [ ]:
onReplica(1)
read(mv_key)